# **Building a Retrieval Augmented Generation (RAG) Based AI Chatbot**

## **1. Introduction to RAG**

Retrieval Augmented Generation (RAG) is a powerful architectural pattern for building AI chatbots and other language model applications that can leverage external knowledge sources to provide more accurate, up-to-date, and contextually relevant responses.

Traditional Large Language Models (LLMs) are trained on vast datasets, but their knowledge is static (frozen at the time of training) and they can sometimes "hallucinate" or generate plausible but incorrect information. RAG addresses these limitations by:

1. **Retrieving** relevant information from a specified knowledge base in real-time.  
2. **Augmenting** the user's query with this retrieved information.  
3. **Generating** a response based on both the original query and the augmented context.

This allows the chatbot to access and utilize domain-specific, private, or rapidly changing information that was not part of its original training data.

## **2. Core Components of a RAG System**

A typical RAG system comprises several key components:

* **Knowledge Base (Data Corpus):** This is the collection of documents, data, or information that the chatbot will use to answer questions. It can be a set of text files, PDFs, web pages, database records, FAQs, articles, etc.  
* **Embedding Model:** This model converts text (from the knowledge base and user queries) into numerical representations called "embeddings" or "vectors." These embeddings capture the semantic meaning of the text, allowing for similarity comparisons. Popular choices include models like Sentence-BERT, OpenAI's Ada, or Cohere's embed models.  
* **Vector Database (Vector Store):** This specialized database is designed to store and efficiently query high-dimensional vectors (the embeddings). It allows for fast similarity searches to find the most relevant pieces of information from the knowledge base based on a query embedding. Examples include Pinecone, Weaviate, FAISS, Chroma, Milvus, or even some traditional databases with vector search capabilities.  
* **Retriever:** This component takes the user's query, converts it into an embedding using the embedding model, and then queries the vector database to find the "top-k" most similar document chunks or pieces of information from the knowledge base.  
* **Large Language Model (LLM) \- The Generator:** This is the core generative model (e.g., GPT-3.5/4, Llama, PaLM, Claude) that takes the original user query *and* the retrieved context (from the Retriever) and generates a coherent, natural language response.  
* **Orchestration/Prompting Layer:** This layer manages the overall workflow. It formats the user's query and the retrieved context into a suitable prompt for the LLM, ensuring the LLM understands how to use the provided information to answer the question.

## **3. Workflow of a RAG Chatbot**

The typical interaction flow in a RAG chatbot is as follows:

1. **User Query:** The user asks the chatbot a question.  
2. **Query Embedding:** The user's query is converted into a vector embedding by the embedding model.  
3. **Information Retrieval:**  
   * The retriever uses this query embedding to search the vector database.  
   * The vector database returns the most semantically similar document chunks (contexts) from the knowledge base.  
4. **Context Augmentation & Prompting:**  
   * The retrieved context(s) are combined with the original user query.  
   * This combined information is formatted into a prompt that instructs the LLM to answer the query based on the provided context.  
5. **Response Generation:** The LLM processes the augmented prompt and generates a natural language response.  
6. **User Receives Response:** The chatbot presents the LLM's generated answer to the user.

## **4. Steps to Build a RAG Chatbot**

Building a RAG chatbot involves several key development stages:

### **Step 1: Data Preparation & Loading**

* **Gather Your Data:** Collect all the documents and data sources that will form your knowledge base.  
* **Preprocess Data:** Clean the data (remove irrelevant characters, formatting issues).  
* **Chunking:** This is a crucial step. LLMs have context window limits. Large documents need to be broken down into smaller, manageable, and semantically coherent chunks. The size of these chunks can impact retrieval quality.  
  * **Strategies:** Fixed-size chunking, sentence-based chunking, recursive chunking, or content-aware chunking (e.g., by paragraphs or sections).  
  * **Overlap:** Often, a small overlap between chunks is introduced to ensure semantic continuity is not lost at chunk boundaries.

### **Step 2: Embedding Generation & Storage**

* **Choose an Embedding Model:** Select an embedding model appropriate for your data and language.  
* **Generate Embeddings:** Process each chunk of your knowledge base through the embedding model to create its vector representation.  
* **Set up Vector Database:** Choose and configure a vector database.  
* **Index Embeddings:** Store these embeddings (along with their corresponding text chunks and any metadata) in the vector database, creating an index for efficient searching.

### **Step 3: Implement the Retriever**

* **Query Embedding Function:** Create a function that takes a user query, uses the same embedding model from Step 2, and generates its embedding.  
* **Similarity Search Function:** Implement logic to take the query embedding and perform a similarity search (e.g., cosine similarity, dot product) against the indexed embeddings in the vector database. This function should return the top-k most relevant text chunks.

### **Step 4: Integrate the Large Language Model (Generator)**

* **Choose an LLM:** Select an LLM suitable for your chatbot's desired capabilities and tone (e.g., OpenAI's API, a self-hosted open-source model).  
* **Prompt Engineering:** This is critical. Design a prompt template that effectively combines the user's original query with the retrieved context. The prompt should guide the LLM to:  
  * Answer the question based *only* on the provided context if possible.  
  * Indicate if the answer cannot be found in the context.  
  * Maintain a conversational tone.  
  * Example Prompt Snippet:

```

Context:
{retrieved_document_chunks}

Question: {user_query}

Answer the question based on the context provided above. If the context doesn't contain the answer, say "I don't have enough information to answer that."

```

### **Step 5: Build the Chat Interface & Orchestration**

* **User Interface (UI):** Develop a front-end for users to interact with the chatbot (e.g., a web interface, integration into an existing messaging platform).  
* **Backend Logic (Orchestration):** Create the backend logic that ties all components together:  
  1. Receives user query from the UI.  
  2. Calls the retriever to get relevant context.  
  3. Constructs the prompt for the LLM.  
  4. Sends the prompt to the LLM API.  
  5. Receives the LLM's response.  
  6. Sends the response back to the UI.  
* **Conversation History (Optional but Recommended):** For multi-turn conversations, you might want to include conversation history in the prompt or use techniques to summarize past turns to provide context for follow-up questions.

## **5. Key Considerations and Best Practices**

* **Chunking Strategy:** Experiment with different chunk sizes and overlap. Too small, and you lose context; too large, and you might introduce noise or exceed LLM context limits.  
* **Embedding Model Choice:** The quality of embeddings significantly impacts retrieval. Choose models trained on similar domains or fine-tune them if necessary.  
* **Retrieval Strategy:**  
  * **Top-k:** How many chunks to retrieve? Too few might miss the answer; too many can overwhelm the LLM or dilute the relevant information.  
  * **Similarity Threshold:** Only retrieve chunks above a certain similarity score to filter out irrelevant results.  
  * **Re-ranking:** Sometimes, a secondary, more sophisticated model (a re-ranker) is used to re-order the initial top-k retrieved chunks for better relevance before sending them to the LLM.  
* **Prompt Engineering:** Iteratively refine your prompts. The way you instruct the LLM to use the context is crucial for response quality and preventing hallucinations.  
* **Evaluation:**  
  * **Retrieval Metrics:** Precision, Recall, Mean Reciprocal Rank (MRR) for the retriever.  
  * **Generation Metrics:** BLEU, ROUGE (less ideal for conversational AI), or human evaluation for the generated responses.  
  * **End-to-End Evaluation:** Assess the overall quality, relevance, and factuality of the chatbot's answers. Frameworks like RAGAs can be helpful.  
* **Handling "I don't know":** Ensure your system can gracefully handle queries for which no relevant information exists in the knowledge base, rather than forcing an answer.  
* **Scalability:** Consider the scalability of your vector database and LLM inference as your knowledge base and user traffic grow.  
* **Maintenance & Updates:** Plan how you will update the knowledge base, re-generate embeddings, and re-index the vector database as new information becomes available.  
* **Cost Management:** Using LLM APIs and vector databases can incur costs. Optimize your system for efficiency.

## **6. Benefits of RAG**

* **Reduced Hallucinations:** By grounding responses in retrieved factual data, RAG significantly reduces the likelihood of the LLM making things up.  
* **Access to Current Information:** LLMs can answer questions based on the latest information available in the knowledge base, unlike their static training data.  
* **Domain-Specific Knowledge:** Enables chatbots to become experts in specific domains by feeding them relevant documents (e.g., medical texts, legal documents, internal company policies).  
* **Transparency and Explainability:** It's often possible to cite the source documents used to generate an answer, increasing trust and allowing users to verify information.  
* **Cost-Effective Customization:** Less expensive than fine-tuning an entire LLM for a new domain, as it primarily involves managing and embedding the knowledge base.

## **7. Potential Challenges**

* **Retrieval Quality:** The entire system heavily relies on the retriever's ability to find the truly relevant information. Poor retrieval leads to poor generation.  
* **Chunking Dilemmas:** Finding the optimal chunking strategy can be challenging and data-dependent.  
* **Integration Complexity:** Connecting various components (embedding models, vector DBs, LLMs) requires careful engineering.  
* **Latency:** The multi-step process (embedding, retrieval, generation) can introduce latency. Optimizations are often needed.  
* **Evaluation Complexity:** Evaluating the end-to-end performance of RAG systems comprehensively can be difficult.



The pre-req steps to use the Gemini API key with Python:

1.  **Get API Key:** Go to [aistudio.google.com](https://aistudio.google.com/), sign in, and create/copy your API key (secure it immediately).
2.  **Install Library:** In your terminal, run `pip install -q -U google-generativeai`.
3.  **Set API Key Securely:**
    * **Recommended:** Set it as an environment variable (e.g., `export GOOGLE_API_KEY="YOUR_KEY"` in terminal/shell config).
    * **Colab:** Use Colab Secrets to store `GOOGLE_API_KEY`.
4.  **Configure in Python:** In your script, retrieve the key (e.g., `os.getenv('GOOGLE_API_KEY')` or `userdata.get('GOOGLE_API_KEY')`) and then use `genai.configure(api_key=YOUR_KEY)`.

[Gemini API Key](https://aistudio.google.com/u/1/apikey)

In [ ]:
## set GOOGLE_API_KEY="408123691657" # now defined in .env in my project dir C:\Users\kh-ab\Documents\GitHub\GenAIEngineering-Cohort2
!pip install langchain-text-splitters langchain-community langchain-huggingface langchain-qdrant sentence-transformers qdrant-client numpy scikit-learn rank_bm25 ipywidgets
!pip install -U google-generativeai python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [ ]:
# Load data
# https://www.kaggle.com/datasets/fajobgiua/indian-food-dataset?resource=download
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
from tqdm.auto import tqdm
import json
import pandas as pd
from langchain_core.documents import Document
# Load models and Initialize Vector Store
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import Qdrant  # Qdrant Vector Store Wrapper
import google.generativeai as genai
from dotenv import load_dotenv  # For loading API key from a .env file
import os
load_dotenv()

tqdm.pandas(desc="Generating Documents")

cleaning_pattern = r'[^a-zA-Z0-9]'

columns = ['TranslatedRecipeName', 'TranslatedIngredients',
           'PrepTimeInMins', 'CookTimeInMins', 'TotalTimeInMins', 'Servings',
           'Cuisine', 'Course', 'Diet', 'TranslatedInstructions', 'URL',
           'ComplexityLevel', 'MainIngredient']

doc_columns = ['score', 'page_content',]


def convert_to_doc(row):
    doc = Document(
        page_content=f'''
# Recipe Name: {row['TranslatedRecipeName']}
> URL: {row['URL']}

## Ingredients:

{row['TranslatedIngredients']}

## Instructions:

{row['TranslatedInstructions']}
''',
        metadata={
            'TranslatedRecipeName': row['TranslatedRecipeName'],
            'PrepTimeInMins': row['PrepTimeInMins'],
            'CookTimeInMins': row['CookTimeInMins'],
            'TotalTimeInMins': row['TotalTimeInMins'],
            'Servings': row['Servings'],
            'Cuisine': row['Cuisine'],
            'Course': row['Course'],
            'Diet': row['Diet'],
            'ComplexityLevel': row['ComplexityLevel'],
            'MainIngredient': row['MainIngredient'],
        }
    )

    return doc


def generate_metadata(search_query, llm):
    meta_prompt = f'''
    Given below the user request for queries, create metadata filter dictionary for the search.

    user query: {search_query}

    > provide only and only a simple phrase for the user query, do not add any other information or context.
    > this output will be used to filter the recipes.

    available metadata:
    - 'Cuisine': string: ['Indian', 'Kerala Recipes', 'Oriya Recipes', 'Continental',
        'Chinese', 'Konkan', 'Chettinad', 'Mexican', 'Kashmiri',
        'South Indian Recipes', 'North Indian Recipes', 'Andhra',
        'Gujarati Recipes']
    - 'Diet': string: ['Vegetarian', 'High Protein Vegetarian', 'Non Vegeterian',
        'Eggetarian', 'Diabetic Friendly', 'High Protein Non Vegetarian',
        'Gluten Free', 'Sugar Free Diet', 'No Onion No Garlic (Sattvic)',
        'Vegan']
    - 'ComplexityLevel': string: ['Medium', 'Hard']

    We can do exact match only.

    respond with a valid json dictionary, do not add any other information or context.
    '''

    resp = llm.generate_content(meta_prompt).text
    metadata = json.loads(resp.split(
        'json')[1].strip().split('```')[0].strip())

    return metadata


def rewrite_query(search_query, llm):
    prompt = f'''
    Given below the user request for queries regarding Indian food recipes, rephrase and expand the query to a more search friendly term.

    user query: {search_query}

    > provide only and only a simple phrase for the user query, do not add any other information or context.
    > this output will be used to search the database for recipes.
    '''
    resp = llm.generate_content(prompt).text
    return resp


def break_query(search_query, llm):
    subquery_prompt = f'''
    Given below the user request for queries, break down the query into multiple subqueries.
    user query: {search_query}
    > provide only and only a simple phrase for the user query, do not add any other information or context.
    > this output will be used to search the database for recipes.

    > respond with a valid json array of strings, do not add any other information or context.
    '''

    resp = llm.generate_content(subquery_prompt).text
    subqueries = json.loads(resp.split(
        'json')[1].strip().split('```')[0].strip())
    return subqueries


def rerank_results(
        search_query,
        searched_df,
        reranking_model
):
    new_doc_embeddings = np.array(
        reranking_model.embed_documents(searched_df.page_content)
    )

    query_embedding = np.array(
        reranking_model.embed_query(search_query)
    )

    similarity_scores = cosine_similarity(
        query_embedding.reshape(1, -1),
        new_doc_embeddings
    )
    searched_df['rerank_score'] = similarity_scores[0].tolist()
    return searched_df


def search(
        search_query,
        llm,
        vector_store,
        reranking_model,
        n_results=10,
        similarity_threshold=0.1,
        flag_rewrite_query=True,
        flag_ai_metadata=True,
        flag_break_query=True,
        flag_rerank_results=True,
):
    """
    Search for the given query in the vector store and return the top n results.
    """
    metadata = {}  # Empty metadata
    subqueries = [search_query]

    if flag_rewrite_query:
        search_query = rewrite_query(search_query, llm)

    if flag_ai_metadata:
        metadata = generate_metadata(search_query, llm)

    if flag_break_query:
        subqueries = break_query(search_query, llm)

    ret_docs = []

    for subquery in subqueries:
        ret_docs += vector_store.similarity_search_with_score(
            subquery,
            k=n_results,
            score_threshold=similarity_threshold,
            filter=metadata
        )

    searched_df = pd.DataFrame(
        [
            {
                'score': score,
                **doc.metadata,
                'page_content': doc.page_content,
            } for doc, score in ret_docs
        ],
        columns=doc_columns+columns
    )

    searched_df = searched_df.groupby(
        'TranslatedRecipeName').first().reset_index()
    searched_df['rerank_score'] = searched_df['score']

    if flag_rerank_results:
        searched_df = rerank_results(
            search_query,
            searched_df=searched_df,
            reranking_model=reranking_model,
        ).sort_values(
            'rerank_score',
            ascending=False,
        )

    return searched_df.head(n_results).round(2)[
        [
            'TranslatedRecipeName',
            'page_content',
            'PrepTimeInMins',
            'CookTimeInMins',
            'TotalTimeInMins',
            'Servings',
            'Cuisine',
            'Diet',
            'ComplexityLevel',
            'MainIngredient',
            'score',
            'rerank_score'
        ]
    ]


def as_cards(df):
    return df.apply(lambda x: x.to_markdown(), axis=1).to_list()


model_name = 'gemini-2.0-flash'

In [ ]:
# Initialize system
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)
print (api_key)


In [ ]:

model = genai.GenerativeModel(model_name)

df = pd.read_csv('./IndianFoodDataset.csv', ).set_index('Srno')[columns]


data = df[:].progress_apply(convert_to_doc, axis=1)


model_768 = HuggingFaceEmbeddings(
    model_name="sentence-transformers/LaBSE",
)

model_384 = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)

model_64 = HuggingFaceEmbeddings(
    model_name="ClovenDoug/tiny_64_all-MiniLM-L6-v2",
)


vector_store_unchunked = Qdrant.from_documents(
    data,
    model_384,
    collection_name="indian-food-metadata",
    location=':memory:',
)

Generating Documents:   0%|          | 0/6871 [00:00<?, ?it/s]

In [ ]:
search_query = 'Healthy Non Veg, Non Spicy, Easy to Cook, 30 Mins, 2 Servings'
results = search(
    search_query,
    llm=model,
    reranking_model=model_768,
    vector_store=vector_store_unchunked,
    n_results=5,
    similarity_threshold=0.5,
    flag_rewrite_query=False,
    flag_ai_metadata=False,
    flag_break_query=False,
    flag_rerank_results=False,
)

results

,TranslatedRecipeName,page_content,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Diet,ComplexityLevel,MainIngredient,score,rerank_score
0,Dal Vangi Recipe,\n# Recipe Name: Dal Vangi Recipe\n> URL: htt...,10,45,55,4,Maharashtrian Recipes,High Protein Vegetarian,Hard,dal,0.64,0.64
1,Goan Masoorchi Usali Recipe,\n# Recipe Name: Goan Masoorchi Usali Recipe\n...,10,35,45,2,Goan Recipes,High Protein Vegetarian,Hard,lentils,0.62,0.62
2,Udupi Style Mixed Vegetable Sambar Recipe (Len...,\n# Recipe Name: Udupi Style Mixed Vegetable S...,20,30,50,4,Mangalorean,Vegetarian,Hard,Dal),0.62,0.62
3,Veg Chilli Milli Recipe,\n# Recipe Name: Veg Chilli Milli Recipe\n> UR...,20,30,50,4,North Indian Recipes,Vegetarian,Hard,shredded,0.63,0.63
4,Vermicelli Biryani (Recipe in Hindi),\n# Recipe Name: Vermicelli Biryani (Recipe in...,15,15,30,4,Indian,Vegetarian,Hard,roasted,0.63,0.63


In [ ]:
search_query = 'Healthy Non Veg, Non Spicy, Easy to Cook, 30 Mins, 2 Servings'
results = search(
    search_query,
    llm=model,
    reranking_model=model_768,
    vector_store=vector_store_unchunked,
    n_results=5,
    similarity_threshold=0.5,
    flag_rewrite_query=True,
    flag_ai_metadata=True,
    flag_break_query=True,
    flag_rerank_results=True,
)

results

,TranslatedRecipeName,page_content,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Diet,ComplexityLevel,MainIngredient,score,rerank_score
0,Chinese Chicken Fried Rice Recipe,\n# Recipe Name: Chinese Chicken Fried Rice R...,10,10,20,2,Chinese,Non Vegeterian,Medium,हुए,0.57,0.42
4,Tempura Chicken Wings With Barbecue Sauce Recipe,\n# Recipe Name: Tempura Chicken Wings With Ba...,10,30,40,4,Continental,Non Vegeterian,Medium,Wings,0.58,0.34
2,Chicken Masala Fry Recipe,\n# Recipe Name: Chicken Masala Fry Recipe\n> ...,5,20,25,1,Indian,Non Vegeterian,Medium,chunks,0.55,0.34
1,Burnt Garlic Chicken Fried Rice Recipe - Indo ...,\n# Recipe Name: Burnt Garlic Chicken Fried Ri...,20,20,40,4,Indo Chinese,Non Vegeterian,Medium,chicken,0.56,0.33
3,Homemade Loaded Chicken Nachos Recipe,\n# Recipe Name: Homemade Loaded Chicken Nacho...,15,10,25,4,Mexican,Non Vegeterian,Medium,pack,0.58,0.26


In [ ]:
user_test_messages = [
    'I want to eat something spicy, non oily and quick to cook.',
    'I am looking for a recipe that is healthy and easy to cook.',
    'I want to eat something that is non vegetarian, quick to cook and spicy.',
    'I am looking for a recipe that is gluten free and easy to cook.',
    'I want to eat something that is diabetic friendly and quick to cook.',
    'I am looking for a recipe that is high protein vegetarian and easy to cook.',
]

In [ ]:
# Ultra Simple Chat Bot
from tqdm.auto import tqdm


prompt = '''You are a friendly and helpful assistant.
Reply to user queries in a friendly and helpful manner, but in the simplest way possible.
Best to keep responses short and to the point.

User Query: {user_message}

Chat History:
{chat_history}

'''

chat_messages = []

for user_message in tqdm(
    user_test_messages,
    desc='simulating conversations'
):
    chat_messages.append({
        'role': 'user',
        'content': user_message,
    })

    prompt = prompt.format(
        user_message=user_message,
        chat_history='\n'.join(
            [
                f"{msg['role']}: {msg['content']}"
                for msg in chat_messages
            ]
        ),
    )

    resp = model.generate_content(prompt).text
    chat_messages.append({
        'role': 'assistant',
        'content': resp,
    })


print(pd.DataFrame([{i['role']: i['content']}
      for i in chat_messages]).fillna('').to_markdown(index=False))

simulating conversations:   0%|          | 0/6 [00:00<?, ?it/s]

| user                                                                        | assistant                                         |
|:----------------------------------------------------------------------------|:--------------------------------------------------|
| I want to eat something spicy, non oily and quick to cook.                  |                                                   |
|                                                                             | How about spicy noodles or stir-fried veggies?    |
| I am looking for a recipe that is healthy and easy to cook.                 |                                                   |
|                                                                             | How about spicy noodles or a stir-fry?            |
| I want to eat something that is non vegetarian, quick to cook and spicy.    |                                                   |
|                                                                           

In [ ]:
# Lets Add in LLM persona and Objective
from tqdm.auto import tqdm

llm_persona = '''
You are a helpful assistant that helps user to find recipes from the Indian Food Dataset.
Your pesona is polite, friendly and helpful.
You are not a chef, you are a recipe search assistant.

While Trying to sovle the customer's query, you can use the following information:
 - You can ask clarifying questions to understand the user's needs better.
 - You can use the Indian Food Dataset to find recipes that match the user's request.
 - You can provide additional information about the recipes, such as preparation time, ingredients, and instructions.
 - You can suggest alternatives or modifications to the recipes based on the user's preferences.

You are not allowed to:
 - Provide medical or dietary advice.
 - Make assumptions about the user's dietary restrictions or preferences without asking.
 - Provide personal opinions or recommendations that are not based on the dataset.
 - Provide recipes that are not in the Indian Food Dataset.
 - Provide recipes that are not relevant to the user's query.
'''

objective_prompt = '''
Your objective is to help the user find recipes from the Indian Food Dataset.
Your hidden side task is also to suggest, cross sell and/upsell dishes.
'''

prompt = '''
Your persona:
{llm_persona}

Your objective:
{objective_prompt}

User Query: {user_message}

Chat History:
{chat_history}

'''

chat_messages = []

for user_message in tqdm(
    user_test_messages,
    desc='simulating conversations'
):
    chat_messages.append({
        'role': 'user',
        'content': user_message,
    })

    prompt = prompt.format(
        llm_persona=llm_persona,
        objective_prompt=objective_prompt,
        user_message=user_message,
        chat_history='\n'.join(
            [
                f"{msg['role']}: {msg['content']}"
                for msg in chat_messages
            ]
        ),
    )

    resp = model.generate_content(prompt).text
    chat_messages.append({
        'role': 'assistant',
        'content': resp,
    })


print(pd.DataFrame([{i['role']: i['content']}
      for i in chat_messages]).fillna('').to_markdown(index=False))

simulating conversations:   0%|          | 0/6 [00:00<?, ?it/s]

| user                                                                        | assistant                                                                                                                                                                                                                                                                                                                                                                                                           |
|:----------------------------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| I want to eat something sp

In [ ]:
# Lets Add in RAG Basic
from tqdm.auto import tqdm

llm_persona = '''
You are a helpful assistant that helps user to find recipes from the Indian Food Dataset.
Your pesona is polite, friendly and helpful.
You are not a chef, you are a recipe search assistant.

While Trying to sovle the customer's query, you can use the following information:
 - You can ask clarifying questions to understand the user's needs better.
 - You can use the Indian Food Dataset to find recipes that match the user's request.
 - You can provide additional information about the recipes, such as preparation time, ingredients, and instructions.
 - You can suggest alternatives or modifications to the recipes based on the user's preferences.

You are not allowed to:
 - Provide medical or dietary advice.
 - Make assumptions about the user's dietary restrictions or preferences without asking.
 - Provide personal opinions or recommendations that are not based on the dataset.
 - Provide recipes that are not in the Indian Food Dataset.
 - Provide recipes that are not relevant to the user's query.
'''

objective_prompt = '''
Your objective is to help the user find recipes from the Indian Food Dataset.
Your hidden side task is also to suggest, cross sell and/upsell dishes.
'''

prompt = '''
Your persona:
{llm_persona}

Your objective:
{objective_prompt}

User Query: {user_message}

Chat History:
{chat_history}

Searched Context from Database:
{context}
'''

rag_prompt = '''
User Query: {user_message}
Chat History:
{chat_history}
'''

chat_messages = []

for user_message in tqdm(
    user_test_messages,
    desc='simulating conversations'
):
    chat_messages.append({
        'role': 'user',
        'content': user_message,
    })

    chat_history = '\n'.join(
        [
            f"{msg['role']}: {msg['content']}"
            for msg in chat_messages
        ]
    )

    results = search(
        rag_prompt.format(
            user_message=user_message,
            chat_history=chat_history,
        ),
        llm=model,
        vector_store=vector_store_unchunked,
        reranking_model=model_768,
        n_results=5,
        similarity_threshold=0.5,
        flag_rewrite_query=False,
        flag_ai_metadata=False,
        flag_break_query=False,
        flag_rerank_results=False,
    )
    context = '\n---\n'.join(as_cards(results))

    prompt = prompt.format(
        llm_persona=llm_persona,
        objective_prompt=objective_prompt,
        user_message=user_message,
        chat_history=chat_history,
        context=context,
    )

    resp = model.generate_content(prompt).text
    chat_messages.append({
        'role': 'assistant',
        'content': resp,
    })


print(pd.DataFrame([{i['role']: i['content']}
      for i in chat_messages]).fillna('').to_markdown(index=False))

simulating conversations:   0%|          | 0/6 [00:00<?, ?it/s]

| user                                                                        | assistant                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|:----------------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Lets Add in RAG Advanced Ultra Pro Max Plus Plus
from tqdm.auto import tqdm

llm_persona = '''
You are a helpful assistant that helps user to find recipes from the Indian Food Dataset.
Your pesona is polite, friendly and helpful.
You are not a chef, you are a recipe search assistant.

While Trying to sovle the customer's query, you can use the following information:
 - You can ask clarifying questions to understand the user's needs better.
 - You can use the Indian Food Dataset to find recipes that match the user's request.
 - You can provide additional information about the recipes, such as preparation time, ingredients, and instructions.
 - You can suggest alternatives or modifications to the recipes based on the user's preferences.

You are not allowed to:
 - Provide medical or dietary advice.
 - Make assumptions about the user's dietary restrictions or preferences without asking.
 - Provide personal opinions or recommendations that are not based on the dataset.
 - Provide recipes that are not in the Indian Food Dataset.
 - Provide recipes that are not relevant to the user's query.
'''

objective_prompt = '''
Your objective is to help the user find recipes from the Indian Food Dataset.
Your hidden side task is also to suggest, cross sell and/upsell dishes.
'''

prompt = '''
Your persona:
{llm_persona}

Your objective:
{objective_prompt}

User Query: {user_message}

Chat History:
{chat_history}

Searched Context from Database:
{context}
'''

rag_prompt = '''
User Query: {user_message}
Chat History:
{chat_history}

'''

chat_messages = []

for user_message in tqdm(
    user_test_messages,
    desc='simulating conversations'
):
    chat_messages.append({
        'role': 'user',
        'content': user_message,
    })

    chat_history = '\n'.join(
        [
            f"{msg['role']}: {msg['content']}"
            for msg in chat_messages
        ]
    )

    results = search(
        rag_prompt.format(
            user_message=user_message,
            chat_history=chat_history,
        ),
        llm=model,
        vector_store=vector_store_unchunked,
        reranking_model=model_768,
        n_results=5,
        similarity_threshold=0.1,
        flag_rewrite_query=True,
        flag_ai_metadata=True,
        flag_break_query=True,
        flag_rerank_results=True,
    )
    context = '\n---\n'.join(as_cards(results))

    resp = model.generate_content(
        prompt.format(
            llm_persona=llm_persona,
            objective_prompt=objective_prompt,
            user_message=user_message,
            chat_history=chat_history,
            context=context,
        )
    ).text
    chat_messages.append({
        'role': 'assistant',
        'content': resp,
    })


print(pd.DataFrame([{i['role']: i['content']}
      for i in chat_messages]).fillna('').to_markdown(index=False))

simulating conversations:   0%|          | 0/6 [00:00<?, ?it/s]

| user                                                                        | assistant                                                                                                                                                                                                                                                                                                                                                                           |
|:----------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| I want to eat something spicy, non oily and quick to cook.                  |             

## **Evaluating Retrieval Augmented Generation (RAG) Systems**

Evaluating Retrieval Augmented Generation (RAG) systems is complex because it requires assessing two distinct stages: the quality of the information retrieved and the quality of the answer generated based on that retrieved context. Poor performance in either stage can lead to unsatisfactory results.

**Key Evaluation Metrics: The RAG Triad**

A common framework for RAG evaluation revolves around the "RAG Triad," which focuses on:

1. **Faithfulness/Groundedness:** This metric assesses if the generated answer is factually supported by the retrieved context. An unfaithful answer, even if fluent, is a hallucination if it contradicts or isn't mentioned in the provided documents.  
   * *Question:* Does the answer stick to the provided context?  
2. **Answer Relevancy:** This measures how relevant the generated answer is to the original user query. An answer can be faithful to the context but still not properly address the user's question.  
   * *Question:* Does the answer directly address the user's question?  
3. **Context Precision & Recall:**  
   * **Context Precision:** Evaluates whether the retrieved context chunks are relevant to the user's query. Irrelevant context can confuse the LLM and lead to poor answers.  
     * *Question:* Is the retrieved information relevant to the question?  
   * **Context Recall:** Assesses if all necessary information required to answer the query was successfully retrieved from the knowledge base. Missing crucial context will likely result in an incomplete or incorrect answer.  
     * *Question:* Was all the necessary information retrieved?

**Evaluation Frameworks & LLM-as-Judge**

Several open-source frameworks help automate RAG evaluation, often using powerful LLMs as "judges" to score the quality of retrieval and generation against the metrics above:

* **Ragas:** Focuses on component-wise evaluation of RAG pipelines using the RAG Triad and other metrics like context relevancy.  
* **DeepEval:** Provides a Pytest-like framework for evaluating LLM applications, including RAG, using various metrics and LLM-based evaluation.  
* **TruLens:** Offers tools for tracking and evaluating LLM experiments, including RAG systems, with a focus on explainability and identifying failure points.

These frameworks typically require a ground truth dataset (questions and ideal answers/contexts) or can perform reference-free evaluation using LLMs to assess quality.

**Typical Evaluation Process:**

1. **Create/Curate Evaluation Datasets:** Develop a set of representative questions, and ideally, corresponding ground truth answers and relevant document contexts.  
2. **Run RAG Pipeline:** Process the evaluation questions through your RAG system to generate answers and log retrieved contexts.  
3. **Compute Metrics:** Use an evaluation framework (like Ragas) or custom scripts to calculate faithfulness, answer relevancy, context precision/recall, and other relevant metrics.  
4. **Analyze Results:** Examine the scores to identify weaknesses in either the retrieval or generation components. For instance, low context recall might indicate issues with your embedding model or retrieval strategy, while low faithfulness might point to problems with your LLM's prompting or its ability to synthesize information accurately.  
5. **Iterate:** Based on the analysis, refine your RAG system (e.g., improve chunking, change embedding models, adjust prompts, fine-tune the LLM) and repeat the evaluation process.

By systematically evaluating both retrieval and generation, developers can build more accurate, reliable, and trustworthy RAG systems.


In [ ]:
# LLM as a Judge

# print(chat_history)

all_user_context = '\n'.join(user_test_messages)

_query = rewrite_query(
    all_user_context,
    model
)

print(_query)

results = search(
    all_user_context,
    model,
    vector_store_unchunked,
    model_768,
    n_results=50,
    similarity_threshold=0.0,
    flag_rewrite_query=True,
    flag_ai_metadata=True,
    flag_break_query=True,
    flag_rerank_results=True,
)
results

Spicy Non-Oily Quick & Healthy Indian Recipes



,TranslatedRecipeName,page_content,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Diet,ComplexityLevel,MainIngredient,score,rerank_score
29,Spicy Cabbage Rice Recipe,\n# Recipe Name: Spicy Cabbage Rice Recipe\n>...,20,45,65,5,Indian,Vegetarian,Hard,हुए,0.67,0.40
24,Quick and Easy Bread Upma (Recipe In Hindi),\n# Recipe Name: Quick and Easy Bread Upma (R...,15,30,45,4,Indian,Vegetarian,Hard,Bread,0.61,0.39
58,Khichdi Roti रेसिपी - खिचड़ी रोटी (Recipe In Hi...,\n# Recipe Name: Khichdi Roti रेसिपी - खिचड़ी र...,0,15,15,4,Indian,Vegetarian,Easy,pre-made,0.58,0.39
95,Sweet Potato & Neem Leaves Vegetable Curry (Re...,\n# Recipe Name: Sweet Potato & Neem Leaves Ve...,20,30,50,4,Indian,Vegetarian,Hard,potatoes,0.59,0.37
27,Savory Flattened Rice & Potato Breakfast (Rec...,\n# Recipe Name: Savory Flattened Rice & Pota...,20,30,50,4,Indian,Vegetarian,Hard,पोहा,0.61,0.36
32,Sweet & Spicy Coriander Tadka Raita (Recipe I...,\n# Recipe Name: Sweet & Spicy Coriander Tadk...,15,15,30,2,Indian,Vegetarian,Medium,ले,0.68,0.36
25,Quinoa Vangi Bath Recipe,\n# Recipe Name: Quinoa Vangi Bath Recipe\n> ...,20,35,55,1,Indian,Vegetarian,Hard,kenua,0.57,0.35
22,No Onion No Garlic Raw Tomato Sabzi (Recipe I...,\n# Recipe Name: No Onion No Garlic Raw Tomat...,10,20,30,6,Indian,Vegetarian,Medium,raw,0.61,0.35
91,Spicy Indian Style Onion Rings Recipe,\n# Recipe Name: Spicy Indian Style Onion Ring...,15,30,45,4,Indian,Vegetarian,Hard,rounds,0.71,0.35
30,Spicy Matar Masala (Recipe In Hindi),\n# Recipe Name: Spicy Matar Masala (Recipe I...,30,30,60,4,Indian,Vegetarian,Hard,boil,0.70,0.35


In [ ]:
context = '\n---\n'.join(as_cards(results))


resp = model.generate_content(
    prompt.format(
        llm_persona=llm_persona,
        objective_prompt=objective_prompt,
        user_message=user_message,
        chat_history=chat_history,
        context=context,
    )
).text
print(resp)

Okay, I understand you're looking for a recipe that is high in protein, vegetarian, and easy to cook. Let's see what the Indian Food Dataset offers!

Based on your requirements, here are a few options for you:

1. **Quinoa Vangi Bath Recipe:** This recipe uses quinoa, which is a good source of protein, along with eggplant and spices. This recipe takes about 55 minutes to cook and is hard to make.
2. **Spicy Cabbage Rice Recipe:** Cabbage Rice with spices and Soya chunks to increase the protein levels is an ideal option. This recipe uses ghee or oil. The total time for this dish is around 65 minutes and complexity level is hard.
3. **Cheesy & Spicy Pull Apart Bread Recipe With Indian Spices:** Pull apart bread filled with potatoes, baby corn, green pepper and soya chunks is another option to explore. However, this recipe takes about 110 minutes to cook and is hard to make.
4.  **Aloo Poha Recipe:** Aloo Poha with green mung sprouts and peanuts is also a good option. This recipe takes ab

In [ ]:
# Evaluating the LLM response using LLM model as a Judge

judge_prompt = '''
You are a judge that evaluates the LLM response.
Your persona is polite, friendly and helpful.
Your task is to evaluate the LLM response and provide feedback.
Your evaluation should be based on the following criteria:
- Relevance: How relevant is the response to the user query?
- Clarity: How clear is the response?
- Completeness: How complete is the response?
- Usefulness: How useful is the response?
- Creativity: How creative is the response?

Candidate LLM response:

{llm_response}

User Context:

{context}

Knowledge base used:

{knowledge_base}

Your evaluation should be in the following format(all scores out of 10):
- Relevance: <relevance_score>/10
- Clarity: <clarity_score>/10
- Completeness: <completeness_score>/10
- Usefulness: <usefulness_score>/10
- Creativity: <creativity_score>/10
- Overall Score: <overall_score>/10
- Feedback: <feedback>
- Suggestions: <suggestions>
- Improvements: <improvements>
- Additional Notes: <additional_notes>
'''

eval_resp = model.generate_content(
    judge_prompt.format(
        llm_response=resp,
        context=all_user_context,
        knowledge_base=context,
    )
).text

print(eval_resp)

- Relevance: 10/10
- Clarity: 9/10
- Completeness: 10/10
- Usefulness: 10/10
- Creativity: 9/10
- Overall Score: 9.6/10
- Feedback: Overall a very good response and recipe suggestions based on the user's request. It understood what the user was asking for and provided good recipe suggestions.
- Suggestions: No suggestions.
- Improvements: No improvements needed.
- Additional Notes: None.



## **Responsible AI: Implementing Guardrails for LLMs**

Large Language Models (LLMs) offer powerful capabilities but also present risks like generating harmful, biased, or incorrect outputs. **Guardrails** are essential mechanisms to guide and control LLM behavior, ensuring safety, reliability, and ethical alignment.

**Why Guardrails are Needed:**

* **Safety & Harm Prevention:** To counter toxicity, bias, misinformation, and harmful instructions.  
* **Behavior Control & Alignment:** To mitigate hallucinations, ensure on-topic responses, maintain brand voice, and prevent prompt injection.  
* **Reliability & Trust:** To improve factual accuracy, ensure compliance with policies, and create predictable behavior.

Without robust guardrails, deploying LLMs risks reputational damage, legal issues, and loss of user trust.

**Types of Guardrails:**

Guardrails operate at different stages of LLM interaction:

1. **Input Guardrails:** Act on user input *before* LLM processing. They block toxic prompts, neutralize prompt injection, check topic relevance, and can anonymize PII. Techniques include filtering, sentiment analysis, and toxicity classifiers.  
2. **Output Guardrails:** Act on the LLM's response *before* user presentation. They filter harmful content, flag hallucinations, ensure topic adherence, and verify against PII disclosure. Techniques include classifiers, fact-checking, and self-critique prompts.  
3. **Dialog Guardrails:** Manage conversation flow and coherence, preventing derailing and ensuring contextual consistency. Techniques involve state machines and topic tracking.  
4. **Retrieval Guardrails (for RAG systems):** Ensure retrieved information is relevant, safe, and trustworthy. Techniques include relevance ranking and metadata filtering.  
5. **Execution Guardrails (for LLMs with action capabilities):** Prevent harmful actions and ensure API calls are authorized and validated. Techniques include whitelisting and human-in-the-loop confirmations.

**Practical Integration with Frameworks:**

Implementing guardrails is simplified by frameworks:

* **NVIDIA NeMo Guardrails:** Uses "Colang" to define topical, fact-checking, and safety rails, operating asynchronously and being LLM-agnostic.  
* **Guardrails AI:** Employs "RAIL" (XML-based) to specify output structure, validation rules, and corrective actions, excellent for structured output.

Other considerations include custom implementations, robust prompt engineering, human-in-the-loop reviews, and continuous monitoring and iteration of guardrails.



In [ ]:
import pandas as pd

toxic_df = pd.read_csv('./train.csv')


def relabel(row):
    return ', '.join([column for column in toxic_df.columns if column in row and row[column] == 1])


toxic_df['label'] = toxic_df.apply(relabel, axis=1)

toxic_df.groupby(['label'])['id'].count().reset_index()

,label,id
0,,143346
1,identity_hate,54
2,insult,301
3,"insult, identity_hate",28
4,obscene,317
5,"obscene, identity_hate",3
6,"obscene, insult",181
7,"obscene, insult, identity_hate",18
8,"obscene, threat",2
9,"obscene, threat, insult",2


In [ ]:
test_label = 'toxic, obscene, threat, insult, identity_hate'

# toxic_df[toxic_df['label'] == test_label].comment_text.apply(len)

comment_text = toxic_df.iloc[88173].comment_text

prompt = f'''
Given the below comment_text, I need you to be responsible AI assistant and
rewrite the comment_text to be decent and acceptable in any forum and decorum
You still need to keep the meaning of the comment_text intact.
Do not add any other information or context.
Ensure that the comment is not harmful, hateful, or abusive in any way.

Output expected:
- Cleaned Text
- Removed Text
- Removed Text Reason

nit: Apply Word Wrapping (80-100 characters max per line) and Sentence Splitting to all output
comment_text: {comment_text}
'''

resp = model.generate_content(prompt).text
print(resp)
print('---')
print(comment_text)

Here's a cleaned version of the comment, focusing on removing hateful and abusive language while retaining the core complaint about the user's editing behavior.

**Cleaned Text:**

"I disagree with your editing practices. I believe you are being
inconsistent and incorrect in your edits. You often revert good work
without contributing positively. I have observed a pattern of
inconsistent and incorrect edits in your history. It seems you undo
other people's contributions, even established ones, without
proper justification. I believe you are acting out of spite and
revenge, rather than with the best interests of the project in mind.

Your actions are unproductive and annoying. I urge you to use talk
pages for dispute resolution and consensus-building instead of
simply reverting edits. Please consider the impact of your actions
and contribute constructively to the community. I believe your
behavior is detrimental to the project."

**Removed Text:**

*   All instances of profanity and name